In [1]:
import pyDOE
from scipy import stats
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
from matplotlib import cm
from scipy.stats import mannwhitneyu
import cma
from scipy.optimize import minimize
from scipy.optimize import Bounds
import sys
from sklearn.preprocessing import MinMaxScaler
import pyDOE
import pandas as pd
import numpy as np
import scipy.stats.distributions as dist
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from collections import namedtuple
from sklearn.linear_model import ElasticNet
from RBFN import RBFN

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Helper Functions

In [2]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' Original Function for Optimization '''
def rastrigin_10D(x):
    x = list(x)
    return sum([y**2 - 10*np.cos(2*math.pi*y) + (10*len(x)) for y in x])

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=10, samples=n_obs, criterion='m')
    X1,X2,X3,X4,X5,X6,X7,X8,X9,X10 = lhd[:,0],lhd[:,1],lhd[:,2],lhd[:,3],lhd[:,4],lhd[:,5], lhd[:,6],lhd[:,7],lhd[:,8],lhd[:,9]
    return X1,X2,X3,X4,X5,X6,X7,X8,X9,X10


'''Robust Regularization based on minimax Principle'''
def robust_regularization(X):
    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10 = X
    eps = np.linspace(-2.048,2.048,1000)
    result = np.zeros(eps.shape[0])
    for i in range(eps.shape[0]):
        result[i]= rastrigin_10D(np.array([x1+eps[i],x2+eps[i],x3+eps[i],x4+eps[i],x5+eps[i],x6+eps[i],
                                        x7+eps[i],x8+eps[i],x9+eps[i],x10+eps[i]]))
    return np.max(result[i])

''' Robustness based on a composite function of Mean and STd '''
def composite_robustness(X):
    w=1
    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10 = X
    np.random.seed(0)
    eps = np.random.normal(loc=0.0,scale=0.3413333333333333,size=10000)
    sample_points = np.zeros(eps.shape[0])
    for i in range(eps.shape[0]):
        sample_points[i]= rastrigin_10D(np.array([x1+eps[i],x2+eps[i],x3+eps[i],x4+eps[i],x5+eps[i],x6+eps[i],
                                        x7+eps[i],x8+eps[i],x9+eps[i],x10+eps[i]]))
    sample_mean = np.mean(sample_points)
    variance = np.square(sample_points-sample_mean)
    std = np.sqrt(np.mean(variance))
    std = np.sqrt(np.mean(variance))
    return sample_mean + w * std

''' Generate Training Data using LHD along side the Output of the Robust System'''
def generate_training_data(n_obs):
    X1,X2,X3,X4,X5,X6,X7,X8,X9,X10 = DOE(n_obs)
    X1 = linearscaletransform(X1,range_out=(-5.12,5.12))
    X2 = linearscaletransform(X2,range_out=(-5.12,5.12))
    X3 = linearscaletransform(X3,range_out=(-5.12,5.12))
    X4 = linearscaletransform(X4,range_out=(-5.12,5.12))
    X5 = linearscaletransform(X5,range_out=(-5.12,5.12))
    X6 = linearscaletransform(X6,range_out=(-5.12,5.12))
    X7 = linearscaletransform(X7,range_out=(-5.12,5.12))
    X8 = linearscaletransform(X8,range_out=(-5.12,5.12))
    X9 = linearscaletransform(X9,range_out=(-5.12,5.12))
    X10 = linearscaletransform(X10,range_out=(-5.12,5.12))
    f_evaluation = np.zeros(X1.shape[0])
    for i in range(X1.shape[0]):
        f_evaluation [i] = rastrigin_10D(np.array([X1[i],X2[i],X3[i],X4[i],X5[i],X6[i],X7[i],X8[i],X9[i],X10[i]]))
    f_original = np.zeros(X1.shape[0])
    minimax_original = np.zeros(X1.shape[0])
    composite_original = np.zeros(X1.shape[0])
    for i in range(X1.shape[0]):
        f_original[i] = rastrigin_10D(np.array([X1[i],X2[i],X3[i],X4[i],X5[i],X6[i],X7[i],X8[i],X9[i],X10[i]]))
        minimax_original[i] = robust_regularization(np.array([X1[i],X2[i],X3[i],X4[i],X5[i],X6[i],X7[i],X8[i],X9[i],X10[i]]))
        composite_original[i] = composite_robustness(np.array([X1[i],X2[i],X3[i],X4[i],X5[i],X6[i],X7[i],X8[i],X9[i],X10[i]]))
    train = pd.DataFrame()
    train['X1'] = pd.Series(X1)
    train['X2'] = pd.Series(X2)
    train['X3'] = pd.Series(X3)
    train['X4'] = pd.Series(X4)
    train['X5'] = pd.Series(X5)
    train['X6'] = pd.Series(X6)
    train['X7'] = pd.Series(X7)
    train['X8'] = pd.Series(X8)
    train['X9'] = pd.Series(X9)
    train['X10'] = pd.Series(X10)
    train['Y = F(X1,X2)'] = pd.Series(f_original)
    train['Y = robust_regularization(X1,X2)'] = pd.Series(minimax_original)
    train['Y = composite(X1,X2)'] = pd.Series(composite_original)
    train.to_csv('Training_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return train

''' Generate Test Data using LHD along side the Output of the Robust System'''
def generate_test_data(n_obs):
    test = pd.DataFrame()
    X = pyDOE.lhs(n=10, samples=n_obs, criterion='m')
    for i in range(X.shape[1]):
        X[:,i] = linearscaletransform(X[:,i],range_out=(-5.12,5.12))
        test['X'+str(i+1)] = pd.Series(X[:,i])
    true_minimax = np.zeros(X.shape[0])
    true_composite = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        true_minimax[i] = robust_regularization(X[i,:])
        true_composite[i] = composite_robustness(X[i,:])
    test ['True_Minimax'] = pd.Series(true_minimax)
    test ['True_Composite'] = pd.Series(true_composite)
    test.to_csv('Test_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return test



''' Create Basis Functions '''
def create_function_basis(x):
    x = np.array(x)
    y = np.zeros(len(x))
    z = np.zeros(45)
    for i in range(len(x)):
        y[i] = (x[i] **2)
    t= 0
    for i in range(0,len(x)):
        for j in range(i+1,len(x)):
            z [t] = x[i] * x [j] 
            t+=1
    x = np.append(x,y)
    x = np.append(x,z)
    x = np.atleast_2d(x)
    return x   


''' Create Basis Functions '''
def create_basis_function_train(data):
    for j in range(1,11):
        data ['X'+str(j)+'**2'] = data['X'+str(j)]**2
    for k in range(1,11):
        for l in range(k+1,11):
            data ['X'+str(k)+'X'+str(l)] = data['X'+str(k)] * data ['X'+str(l)]
    f_original = data['Y = F(X1,X2)']
    robust_original = data['Y = robust_regularization(X1,X2)']
    composite_original = data ['Y = composite(X1,X2)']
    del data['Y = F(X1,X2)']
    del data['Y = robust_regularization(X1,X2)']
    del data ['Y = composite(X1,X2)']
    data['Y = F(X1,X2)'] = f_original
    data['Y = robust_regularization(X1,X2)'] = robust_original
    data ['Y = composite(X1,X2)'] = composite_original
    return data

def create_basis_function_test(data):
    for j in range(1,11):
        data ['X'+str(j)+'**2'] = data['X'+str(j)]**2
    for k in range(1,11):
        for l in range(k+1,11):
            data ['X'+str(k)+'X'+str(l)] = data['X'+str(k)] * data ['X'+str(l)]
    true_minimax = np.array(data.True_Minimax)
    true_composite = np.array(data.True_Composite)
    del data['True_Minimax']
    del data ['True_Composite']
    data['True_Minimax'] = pd.Series(true_minimax)
    data ['True_Composite'] = pd.Series(true_composite)
    return data


## Meta-Models

In [3]:
''' Elastic Net Regression for Minimax Robustness'''
def elastic_net_minimax(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:65].values, test_data.values])
    regr=ElasticNet(alpha= 0.009782831659031866 ,random_state=0 , l1_ratio= 0.22322322322322322 , fit_intercept =True, max_iter=1000,selection='random'
                   ).fit(scaler.transform(train_data.iloc[:,:65]), train_data.iloc[:,66])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)

''' Elastic Net Regression for Minimax Robustness'''
def elastic_net_composite(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:65].values, test_data.values])
    regr=ElasticNet(alpha= 0.006493421766031815     ,random_state=0 , l1_ratio= 1.0, fit_intercept =True, max_iter=1000,selection='random'
                   ).fit(scaler.transform(train_data.iloc[:,:65]), train_data.iloc[:,-1])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)


''' Kriging for Robust Regularization'''
def kriging_minimax(train_data,test_data):
    kernel =  RBF( 74, (99, 83935) )
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=93,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,11])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Kriging for Robust Composition'''
def kriging_composite(train_data,test_data):
    kernel =  RBF( 10, (75, 28334) )
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=45,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Random Forest Implementation for Minimax Robustness'''
def rf_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    regr = RandomForestRegressor(random_state=13,n_estimators=91
                                ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,11])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)

''' Random Forest Implementation for Composite Robustness'''
def rf_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    regr = RandomForestRegressor(random_state=37,n_estimators=77
                                ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,-1])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)


''' KNN Regression Implementation for Minimax Robustness'''
def KNN_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    regr = KNeighborsRegressor(n_neighbors=8,weights='distance',algorithm='brute',p=2
                               ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,11])

    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)

''' KNN Regression Implementation for Composite Robustness'''
def KNN_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    regr = KNeighborsRegressor(n_neighbors=4,weights='distance',algorithm='brute',p=2
                               ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,-1])

    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)

''' Support Vector Regression for Robust Regularization'''
def SVR_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    gpr = SVR(gamma =  8.53443233e-02    ,C =  7.03315585e+02 , epsilon=0.2,max_iter=1500).fit(
        scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,11])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression for Robust Composition'''
def SVR_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    gpr = SVR(gamma = 8.53443233e-02 ,C =  7.03315585e+02 , epsilon=0.2,max_iter=1500).fit(
        scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' RBF Network Interpolation for Robust Regularization'''
def RBF_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    model = RBFN(hidden_shape=90, sigma= 0.001)
    model.fit(scaler.transform(np.array(train_data.iloc[:,:10])), np.array(train_data.iloc[:,11]))
    pred = model.predict(scaler.transform(np.array(test_data)))
    def predict(scaler, model):
        def __predict__(x):
            x = np.atleast_2d(x)
            return model.predict(scaler.transform(x))
        return __predict__
    return model,pred, predict(scaler,model)

''' RBF Network Interpolation for Robust Composition'''
def RBF_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    model = RBFN(hidden_shape=90, sigma= 0.001000)
    model.fit(scaler.transform(np.array(train_data.iloc[:,:10])), np.array(train_data.iloc[:,-1]))
    pred = model.predict(scaler.transform(np.array(test_data)))
    def predict(scaler, model):
        def __predict__(x):
            x = np.atleast_2d(x)
            return model.predict(scaler.transform(x))
        return __predict__
    return model,pred, predict(scaler,model)



In [4]:
train = pd.read_csv(sys.path[1]+str('\\Training_Data_Sets\\500Samples.csv')).iloc[:,1:]
test = pd.read_csv(sys.path[1]+str('\\Test_Data_Sets\\750Samples.csv')).iloc[:,1:]
model_k_m,pred_k_m, predict_k_m = kriging_minimax(train,test.iloc[:,:10])
model_k_c,pred_k_c, predict_k_c = kriging_composite(train,test.iloc[:,:10])
model_r_m,pred_r_m, predict_r_m = rf_minimax(train,test.iloc[:,:10])
model_r_c,pred_r_c, predict_r_c = rf_composite(train,test.iloc[:,:10])
model_n_m,pred_n_m, predict_n_m = KNN_minimax(train,test.iloc[:,:10])
model_n_c,pred_n_c , predict_n_c= KNN_composite(train,test.iloc[:,:10])
model_s_m,pred_s_m, predict_s_m = SVR_minimax(train,test.iloc[:,:10])
model_s_c,pred_s_c, predict_s_c = SVR_composite(train,test.iloc[:,:10])
model_b_m,pred_b_m, predict_b_m = RBF_minimax(train,test.iloc[:,:10])
model_b_c,pred_b_c, predict_b_c = RBF_composite(train,test.iloc[:,:10])
train = create_basis_function_train(train)
test = create_basis_function_test(test)
model_m_m,pred_m_m, predict_m_m = elastic_net_minimax(train,test.iloc[:,:65])
model_m_c,pred_m_c, predict_m_c = elastic_net_composite(train,test.iloc[:,:65])

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [5]:
Columns = ['Benchmark', 'Kriging' , 'SVM' , 'RBFN' ,  'KNN' , 'RF'  , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,11):
        Cols.append(Columns[j]+'_X'+str(i))
        
const = Bounds([-5.12,-5.12,-5.12,-5.12,-5.12,-5.12,-5.12,-5.12,-5.12,-5.12], 
               [5.12,5.12,5.12,5.12,5.12,5.12,5.12,5.12,5.12,5.12])

G1,G2,G3,G4,G5,G6,G7,G8,G9,G10 = DOE(100)
G1 = linearscaletransform(G1,range_out=(-5.12,5.12))
G2 = linearscaletransform(G2,range_out=(-5.12,5.12))
G3 = linearscaletransform(G3,range_out=(-5.12,5.12))
G4 = linearscaletransform(G4,range_out=(-5.12,5.12))
G5 = linearscaletransform(G5,range_out=(-5.12,5.12))
G6 = linearscaletransform(G6,range_out=(-5.12,5.12))
G7 = linearscaletransform(G7,range_out=(-5.12,5.12))
G8 = linearscaletransform(G8,range_out=(-5.12,5.12))
G9 = linearscaletransform(G9,range_out=(-5.12,5.12))
G10 = linearscaletransform(G10,range_out=(-5.12,5.12))



X_Values_RR = np.zeros([100,70])
X_Values_RC = np.zeros([100,70])
Fun_Eval_RR = np.zeros([100,7])
Fun_Eval_RC = np.zeros([100,7])
opt = {'maxiter':50}

## SLSQP

In [6]:
for i in range(X_Values_RR.shape[0]):
    min_robust = minimize(robust_regularization,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_compo = minimize(composite_robustness,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_robust_k = minimize(predict_k_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_composite_k = minimize(predict_k_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_robust_s = minimize(predict_s_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_composite_s = minimize(predict_s_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_robust_b = minimize(predict_b_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_composite_b = minimize(predict_b_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_robust_n = minimize(predict_n_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_composite_n = minimize(predict_n_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_robust_r = minimize(predict_r_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_composite_r = minimize(predict_r_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_robust_m = minimize(predict_m_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    min_composite_m = minimize(predict_m_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i],G6[i],G7[i],G8[i],G9[i],G10[i]])),method='SLSQP',bounds=const,options=opt)
    
   
    
    
    
    X_Values_RR [i,:] = list(min_robust.x)+list(min_robust_k.x)+list(min_robust_s.x)+list(min_robust_b.x)+list(min_robust_n.x)+list(min_robust_r.x)+list(min_robust_m.x)
    
    X_Values_RC [i,:] = list(min_compo.x)+list(min_composite_k.x)+list(min_composite_s.x)+list(min_composite_b.x)+list(min_composite_n.x)+list(min_composite_r.x)+list(min_composite_m.x)
    
    Fun_Eval_RR [i,:] = [min_robust.nfev]+[100,100,100,100,100,100]
    
    Fun_Eval_RC [i,:] = [min_compo.nfev]+[100,100,100,100,100,100]

In [7]:
X_Values_RR = pd.DataFrame(X_Values_RR)
X_Values_RR.columns = Cols
X_Values_RC = pd.DataFrame(X_Values_RC)
X_Values_RC.columns = Cols
Fun_Eval_RR = pd.DataFrame(Fun_Eval_RR)
Fun_Eval_RR.columns = Columns
Fun_Eval_RC = pd.DataFrame(Fun_Eval_RC)
Fun_Eval_RC.columns = Columns

In [8]:
X_Values_RR = pd.DataFrame(X_Values_RR)
X_Values_RR.columns = Cols
X_Values_RC = pd.DataFrame(X_Values_RC)
X_Values_RC.columns = Cols
Fun_Eval_RR = pd.DataFrame(Fun_Eval_RR)
Fun_Eval_RR.columns = Columns
Fun_Eval_RC = pd.DataFrame(Fun_Eval_RC)
Fun_Eval_RC.columns = Columns

In [9]:
X_Values_RR.to_csv('Results\\X_Values_RR.csv')
X_Values_RC.to_csv('Results\\X_Values_RC.csv')
Fun_Eval_RR.to_csv('Results\\Function_Eval_RR.csv')
Fun_Eval_RC.to_csv('Results\\Function_Eval_RC.csv')
RR = pd.read_csv('Results\\X_Values_RR.csv').iloc[:,1:]
RC = pd.read_csv('Results\\X_Values_RC.csv').iloc[:,1:]

## Meta-Modelling

In [10]:
Bench_Fun_RR = np.zeros(100)
Krig_Fun_RR = np.zeros(100)
SVM_Fun_RR = np.zeros(100)
RBFN_Fun_RR = np.zeros(100)
KNN_Fun_RR = np.zeros(100)
RF_Fun_RR = np.zeros(100)
ELN_Fun_RR = np.zeros(100)
for i in range(X_Values_RR.shape[0]):
    Bench_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,:10])
    Krig_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,10:20])
    SVM_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,20:30])
    RBFN_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,30:40])
    KNN_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,40:50])
    RF_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,50:60])
    ELN_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,60:70])

In [11]:
print ('SLSQP')
print (stats.mode(Bench_Fun_RR) , np.std(Bench_Fun_RR))
print ('Kriging')
print (stats.mode(Krig_Fun_RR) , np.std(Krig_Fun_RR))
print ('SVM')
print (stats.mode(SVM_Fun_RR) , np.std(SVM_Fun_RR))
print ('RBFN')
print (stats.mode(RBFN_Fun_RR) , np.std(RBFN_Fun_RR))
print ('KNN')
print (stats.mode(KNN_Fun_RR) , np.std(KNN_Fun_RR))
print ('RF')
print (stats.mode(RF_Fun_RR) , np.std(RF_Fun_RR))
print ('ELN')
print (stats.mode(ELN_Fun_RR) , np.std(ELN_Fun_RR))

SLSQP
ModeResult(mode=array([918.90418693]), count=array([1])) 40.0400574218273
Kriging
ModeResult(mode=array([1038.30460493]), count=array([1])) 53.761180727327115
SVM
ModeResult(mode=array([952.48414242]), count=array([1])) 0.02373913100416325
RBFN
ModeResult(mode=array([1033.37983989]), count=array([1])) 49.88780093211833
KNN
ModeResult(mode=array([974.75554433]), count=array([1])) 30.770567825819672
RF
ModeResult(mode=array([1046.84898377]), count=array([1])) 52.62300540136485
ELN
ModeResult(mode=array([1032.21296732]), count=array([1])) 0.021996387059738356


In [12]:
Bench_Fun_RC = np.zeros(100)
Krig_Fun_RC = np.zeros(100)
SVM_Fun_RC = np.zeros(100)
RBFN_Fun_RC = np.zeros(100)
KNN_Fun_RC = np.zeros(100)
RF_Fun_RC = np.zeros(100)
ELN_Fun_RC = np.zeros(100)
for i in range(X_Values_RC.shape[0]):
    Bench_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,:10])
    Krig_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,10:20])
    SVM_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,20:30])
    RBFN_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,30:40])
    KNN_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,40:50])
    RF_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,50:60])
    ELN_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,60:70])

In [13]:
print ('SLSQP')
print (stats.mode(Bench_Fun_RC) , np.std(Bench_Fun_RC))
print ('Kriging')
print (stats.mode(Krig_Fun_RC) , np.std(Krig_Fun_RC))
print ('SVM')
print (stats.mode(SVM_Fun_RC) , np.std(SVM_Fun_RC))
print ('RBFN')
print (stats.mode(RBFN_Fun_RC) , np.std(RBFN_Fun_RC))
print ('KNN')
print (stats.mode(KNN_Fun_RC) , np.std(KNN_Fun_RC))
print ('RF')
print (stats.mode(RF_Fun_RC) , np.std(RF_Fun_RC))
print ('ELN')
print (stats.mode(ELN_Fun_RC) , np.std(ELN_Fun_RC))

SLSQP
ModeResult(mode=array([1003.4000837]), count=array([1])) 12.134756680628389
Kriging
ModeResult(mode=array([1057.55901114]), count=array([1])) 23.840130856618025
SVM
ModeResult(mode=array([1044.6362973]), count=array([1])) 0.017307890156653735
RBFN
ModeResult(mode=array([1040.77422711]), count=array([1])) 23.478400330661987
KNN
ModeResult(mode=array([1023.47266488]), count=array([1])) 25.391563414351623
RF
ModeResult(mode=array([1063.85092953]), count=array([1])) 25.25585447377904
ELN
ModeResult(mode=array([1054.53706032]), count=array([1])) 0.003001863736954622


In [ ]:
11,46 thursday